In [1]:
from sentistrength import PySentiStr
import pandas as pd

senti = PySentiStr()
senti.setSentiStrengthPath('C:/Users/femin/MP_Final/SentiStrength.jar') # Note: Provide absolute path instead of relative path
senti.setSentiStrengthLanguageFolderPath('C:/Users/femin/MP_Final/SentiStrength/') # Note: Provide absolute path instead of relative path


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import ast
ps = PorterStemmer()
import nltk
from nltk.corpus import wordnet as wn

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\femin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def process_tweet(tweet):
    
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'RT', '', tweet)
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = splitTags(tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
#             tweets_clean.append(stem_word)

    return tweets_clean

In [5]:
def splitTags(s):
    l = [i  for i in s.split() if i.startswith("#") ]
    for i in l:
        expanded = " ".join([a for a in re.split('([A-Z][a-z]+)', i[1:]) if a])
        s = s.replace(i,expanded)
    return s

In [6]:

def stemming(processed):
    stemmed = [ps.stem(i) for i in processed if i != '']
    return stemmed

In [7]:
def sentiCalc(tweet):
    scale = senti.getSentiment(tweet, score = 'scale')
    dual = senti.getSentiment(tweet, score = 'dual')
    binary = senti.getSentiment(tweet, score = 'binary')
    return [scale[0],dual[0][0],dual[0][1],binary[0]]

In [8]:
def generate_x(tweet):
    processed = process_tweet(tweet)
    print("\nPreProcessing and Tokenization")
    print(processed)
    stemmed = stemming(processed)
    print("\nStemming")
    print(stemmed)
    sentiment = sentiCalc(tweet)
    print("\nSentiment(Scale,Pos,Neg,Binary)")
    print(sentiment)
    semantics = generate_punctuation_rows(tweet)
    print("\nSemantics (QuestionMarkCount, QuotationMarkCount, ExclamationMarkCount, AllCapitalised) ")
    print(semantics)
    unis = generate_col(stemmed,cols)
    print("\nUnigrams")
    print(unis)
#     print(processed,stemmed,sentiment,unis,semantics)
    return processed,stemmed,sentiment,unis,semantics


In [9]:
cols = []
with open('C:/Users/femin/MP_Final/combined.txt') as f:
    cols = f.read().splitlines()

In [10]:
def generate_col(row,col):
    uni = []
    for i in col:
        if i in row:
            uni.append(1)
        else:
            uni.append(0)
    return uni

In [11]:
def generate_punctuation_rows(tweet):
#     QuestionMarkCount | QuotationMarkCount | ExclamationMarkCount | AllCapitalised
    x, p= process_tweet2(tweet) # need to use process tweet to remove redundant exclamation and RT string
#     print(p)
    res=[]
    if '?' in p: 
        res.append(p['?']) 
    else:
        res.append(0)
    if '\'' in p: 
        res.append(p['\'']) 
    else:
        res.append(0)
    if '!' in p: 
        res.append(p['!']) 
    else: 
        res.append(0)
   
    res.append(p['allCaps']) 
    return res

In [12]:
def count_punctuation_and_allcaps(tweet):
    p={'allCaps':0}
    if type(tweet)==str:
        tweet_str= tweet
    else:
        tweet_str=' '.join(tweet)
#     print(tweet_str)
    temp= re.findall('[A-Z]{2,}', tweet_str)
#     print(temp)
    p['allCaps']=len(temp)
    
    for i in tweet:
        if i in string.punctuation and i not in ['@','#'] and i not in p:
            p[i]=tweet.count(i)
    return p

In [13]:
def process_tweet2(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    tweet= re.sub("!*\sRT?", ' ', tweet)
    tweet= re.sub("\A!*", '',  tweet)
    tweet= re.sub("\A\sRT\s", ' ',  tweet)
    
    x= count_punctuation_and_allcaps(tweet)
    tweet = re.sub(r'RT', '', tweet)
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = splitTags(tweet)
    
# %%%%%%% changed preserve case parameter to True
    tokenizer = TweetTokenizer()
    tweet_tokens = tokenizer.tokenize(tweet)
#     print(tweet,tweet_tokens)
    
#     print(x)
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
#             tweets_clean.append(stem_word)

    return tweets_clean, x

In [14]:
def euphemism(name):
    synonyms = []
    for syn in wn.synsets(name):
        for hyp in syn.hypernyms():
            if wn.wup_similarity(hyp, syn)>0.1:
                for l in hyp.lemmas():
                    synonyms.append(l.name().replace('_',' ')) 
    return synonyms[:]

In [ ]:
# final_x = sentiment[:0] + semantics + unis

In [ ]:
# final_x = np.reshape(final_x,(1,-1))
# print(final_x)

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, cohen_kappa_score

In [16]:
from sklearn.linear_model import LogisticRegression
import pickle

In [17]:
Pkl_Filename = "ternary_model.pkl"  
with open(Pkl_Filename, 'rb') as file:  
    lr = pickle.load(file)

In [ ]:
# y_pred=lr.predict(final_x)

In [ ]:
# print(y_pred)

In [18]:
def final_output(tweet):
    processed,stemmed,sentiment,unis,semantics = generate_x(tweet)
    euphems = generate_euphemisms(processed,stemmed)
    final_x = sentiment + semantics + unis
    final_x = np.reshape(final_x,(1,-1))
    print("\nFinal Input Vector: \n",final_x)
    Pkl_Filename = "ternary_model.pkl"  
    with open(Pkl_Filename, 'rb') as file:  
        lr = pickle.load(file)
    prob = (lr.predict_proba(final_x))[0]
    
    print('\n\nClass Probablities')
    
    [print("{0:.5f}".format(i)) for i in prob]
    y_pred=lr.predict(final_x)

    if y_pred[0] == 0:
        return 'Hateful',euphems
    else :
        return 'Clean',euphems
    

In [22]:
s = ' @Shun_stokes: This is a really good book to read. Are you going to read it?'
classes ,euphem = final_output(s)
print('\n')
for k in euphem:
    print(k,": ",euphem[k])
    print()
# print(euphem)
print('Prediction: ',classes)


PreProcessing and Tokenization
['answer', 'snapchat', 'faggot', 'butt', 'hurt']

Stemming
['answer', 'snapchat', 'faggot', 'butt', 'hurt']

Sentiment(Scale,Pos,Neg,Binary)
[-1, 1, -2, -1]

Semantics (QuestionMarkCount, QuotationMarkCount, ExclamationMarkCount, AllCapitalised) 
[0, 0, 0, 0]

Unigrams
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Final Input Vector: 
 [[-1  1 -2 -1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0 

In [20]:
def generate_euphemisms(processed,stemmed):
    cols = []
    with open('C:/Users/femin/MP_Final/combined.txt') as f:
        cols = f.read().splitlines()
    d = {}
    for i in range(len(processed)):
        if stemmed[i] in cols:
            s = euphemism(processed[i])
            d[processed[i]]=euphemism(processed[i])
    return d

In [ ]:
euphemism('bitch')

- &#8220;@NoChillPaz: White kids favorite activities 1. Calling people "niggers" on Xbox 2.fucking their pet 3. Shooting up their school&#8221;

- @AustinG1135 answer my snapchat faggot #ButtHurt

- @IGGYAZALEA it was a joke you fugly bitch be happy uncle snoop even acknowledged your nasty ass lmao

- @Shun_stokes: My cousin said he share bitches with his brothers. He said sharing is caring and he love his brothers and the sluts &#128514'

- @MiAmor Yo he just a retarded spic, tell that nigga to go fuck himself